In [ ]:
import pygame,sys,random    #sys is used for ending a program

def draw_floor():
    screen.blit(floor_surface,(floor_x_position,710))
    screen.blit(floor_surface,(floor_x_position+576,710))
    
    
    
def create_pipe():
    random_pipe_position=random.choice(pipe_height)      #we will choose random number from the pipe height list
    bottom_new_pipe=pipe_surface.get_rect(midtop=(700,random_pipe_position))
    top_new_pipe=pipe_surface.get_rect(midbottom=(700,random_pipe_position-250))
    return bottom_new_pipe,top_new_pipe

def move_pipes(pipes):
    for pipe in pipes:
        pipe.centerx -=5
    return pipes


def draw_pipes(pipes):
    
    for pipe in pipes:
        if pipe.bottom>=800:
             screen.blit(pipe_surface,pipe)
        else:
            flip_pipe=pygame.transform.flip(pipe_surface, False,True)
            screen.blit(flip_pipe,pipe)
            
def check_collision(pipes):  
     
    for pipe in pipes:
        
        if  bird_rect.colliderect(pipe):
            death_sound.play()
            return False
            
            
    if bird_rect.top <=-80  or bird_rect.bottom >=  710:
        return False
   
            
   
    return True    
            
    
def rotate_bird(bird):
    new_bird=pygame.transform.rotozoom(bird,-bird_movement*3,1)
    return new_bird


def bird_animation():
    new_bird=bird_frames[bird_index]
    new_bird_rect=new_bird.get_rect(center=(100,bird_rect.centery))
    return new_bird,new_bird_rect
    

    
    
def score_display(game_state):
    if game_state == 'main_game':
        
        score_surface = game_font.render(str(int(score)),True,(255,255,255))         #RGB colours composition
        score_rect=score_surface.get_rect(center=(288,100))                  #making a rect around our score board
        screen.blit(score_surface,score_rect)

    if game_state == 'game_over':
        
        score_surface = game_font.render(f'Score:{int(score)}',True,(255,255,255))         #RGB colours composition
        score_rect=score_surface.get_rect(center=(288,100))                  #making a rect around our score board
        screen.blit(score_surface,score_rect)
        
        
        high_score_surface = game_font.render(f' High score:{int(high_score)}',True,(255,255,255))         #RGB colours composition
        high_score_rect=high_score_surface.get_rect(center=(288,690))                  #making a rect around our score board
        screen.blit(high_score_surface,high_score_rect)
        
    
def update_score(score,high_score) :
    if score > high_score:
        high_score = score
        
    return high_score    
    

    
    
pygame.mixer.pre_init(frequency=44100,size=16,channels=1,buffer=800)    
pygame.init()    #initiates pygame

screen= pygame.display.set_mode((576,800))  #width=576,height=1074
clock= pygame.time.Clock()  #used for setting frame rate
game_font = pygame.font.SysFont('ARIAL',36)

#GAME VARIABLES

gravity=0.35
bird_movement=0
game_active = True
score= 0
high_score = 0



bg_surface=pygame.image.load("assets/background-day.png").convert_alpha()    #we just imported this but not used in background
bg_surface=pygame.transform.scale2x(bg_surface )                       #we transformed our background image and doubled it in size 



floor_surface=pygame.image.load("assets/base.png").convert()
floor_surface=pygame.transform.scale2x(floor_surface)
floor_x_position=0



bird_downflap= pygame.transform.scale2x(pygame.image.load("assets/bluebird-downflap.png")).convert_alpha()
bird_midflap= pygame.transform.scale2x(pygame.image.load("assets/bluebird-midflap.png")).convert_alpha()
bird_upflap= pygame.transform.scale2x(pygame.image.load("assets/bluebird-upflap.png")).convert_alpha()
bird_frames=[bird_downflap,bird_midflap,bird_upflap]
bird_index=0
bird_surface=bird_frames[bird_index]
bird_rect= bird_surface.get_rect(center=(100,400))

BIRDFLAP= pygame.USEREVENT +1
pygame.time.set_timer(BIRDFLAP,200)



#bird_surface=pygame.image.load("assets/bluebird-midflap.png").convert()
#bird_surface=pygame.transform.scale2x(bird_surface)
#bird_rect=bird_surface.get_rect(center=(100,400))

pipe_surface=pygame.image.load("assets/pipe-green.png").convert()
pipe_surface=pygame.transform.scale2x(pipe_surface)
pipe_list=[]
SPAWNPIPE=pygame.USEREVENT
pygame.time.set_timer(SPAWNPIPE,1200)            #1200 milliseconds means 1.2 sec



pipe_height=(300,450,580)



game_over_surface=pygame.transform.scale2x(pygame.image.load("assets/message.png")).convert_alpha()
game_over_rect=game_over_surface.get_rect(center=(288,400))


flap_sound=pygame.mixer.Sound('sound/sfx_wing.wav')
death_sound=pygame.mixer.Sound('sound/sfx_hit.wav')
score_sound=pygame.mixer.Sound('sound/sfx_point.wav')
score_sound_countdown=100
while True:
    
    
    
    for event in pygame.event.get():               #catches all the movements of mouse or pressing keys
        
        if event.type == pygame.QUIT:
            pygame.quit() 
            sys.exit()                             #helps to shutdown our program 
            
        if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE and game_active:   
                    bird_movement=0
                    bird_movement-=12
                    flap_sound.play()
                if event.key == pygame.K_SPACE  and game_active==False:     # and active_game==False
                    game_active= True
                    pipe_list.clear()
                    bird_rect.center=(100,400)
                    bird_movement=0
                    score = 0
                    
                    
        if event.type==SPAWNPIPE:
            pipe_list.extend(create_pipe())
            
            
        if event.type==BIRDFLAP:
            if bird_index<2:
                bird_index+=1
            else:
                bird_index=0
                
                
            bird_surface ,bird_rect=bird_animation()   
                
                
    
    screen.blit(bg_surface,(0,0))
    
     
   
    if game_active:   
            
             #BIRD
        bird_movement +=gravity                 #bird_movement increases 
        rotated_bird=rotate_bird(bird_surface)
        bird_rect.centery +=bird_movement
        screen.blit(rotated_bird,bird_rect)
        check_collision(pipe_list)
        game_active = check_collision(pipe_list)
    
       #PIPES
        pipe_list= move_pipes(pipe_list)
        draw_pipes(pipe_list)
        
        #score
        
        score +=0.01
        score_display('main_game') 
        score_sound_countdown -=1
        if score_sound_countdown <= 0:
            score_sound.play()
            score_sound_countdown =100
        
    else:
        
        
        
        high_score = update_score(score,high_score)
        score_display('game_over')
        screen.blit(game_over_surface,game_over_rect)
            
   
   
    
    
    
    #FLOOR
    floor_x_position -=2                      #position of floor changes by 2
    draw_floor()
    if floor_x_position<= -576 :               #whenever the flow gets towards the extreme left(-576) then it restarts from 0
        floor_x_position=0
    
    
    
    
    pygame.display.update()                   #displays items from the while loop
    clock.tick(120)                           #setting frame rate to 120 fps



pygame 2.0.2 (SDL 2.0.16, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html
